In [10]:
#pip install langchain-groq langchain-groq
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [3]:
llm = ChatGroq(
    temperature=0.5,
    model_name="llama3-8b-8192"
)

In [4]:
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text in a concise paragraph:\n\n{text}"
)

In [5]:
translate_prompt = PromptTemplate(
    input_variables=["summary", "language"],
    template="Translate the following summary into {language}:\n\n{summary}"
)


In [6]:
summarize_chain = LLMChain(llm=llm, prompt=summary_prompt)


C:\Users\moham\AppData\Local\Temp\ipykernel_15576\4278470831.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summarize_chain = LLMChain(llm=llm, prompt=summary_prompt)


In [7]:
translate_chain = LLMChain(llm=llm, prompt=translate_prompt)


In [ ]:
def summarize_and_translate(text, target_language):
    # Summarize
    summary = summarize_chain.run(text)
    print("🔍 Summary:")
    print(summary)

    # Translate
    translation = translate_chain.run({
        "summary": summary,
        "language": target_language
    })
    print("\n🌐 Translation:")
    print(translation)


In [ ]:
input_text = """
    Artificial Intelligence (AI) has revolutionized education in recent years.
    From personalized tutoring systems and automated grading to intelligent content recommendations,
    AI enables teachers to focus more on creative and interpersonal tasks.
    However, concerns remain about bias in algorithms and student data privacy.
    """

target_language = "Tamil"  # "Hindi", "German", "Tamil"
summarize_and_translate(input_text, target_language)

🔍 Summary:
Artificial Intelligence (AI) has significantly impacted education, allowing teachers to focus on creative and interpersonal tasks by automating tasks such as personalized tutoring, grading, and content recommendations. While AI has brought numerous benefits, concerns persist regarding bias in algorithms and the protection of student data privacy.

🌐 Translation:
கல்வி துறையில் கல்வி நிபந்தனைகள் (AI) பெற்ற பெரிய பாதிப்புகளைக் கொண்டுள்ளது. கல்வியாளர்கள் பட்டியலிட்ட கல்வி, மதிப்பெண் வகிப்பு, உள்ளடக்க பரிந்துரைகள் போன்ற செயல்களை தானியக்க முடியும் என்றாலும், கணினி நிபந்தனைகளில் சமானத்தையும் மாணவர்களின் தகவல் பொறுப்புக்காக கவலைகள் உள்ளன.


In [ ]:
# GUI Implementation
import gradio as gr
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

os.environ["GROQ_API_KEY"] = "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"

# Initialize Groq LLaMA3 LLM
llm = ChatGroq(
    temperature=0.5,
    model_name="llama3-8b-8192"
)

# Prompt to summarize the input text
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text in a concise paragraph:\n\n{text}"
)

# Prompt to translate the summary
translate_prompt = PromptTemplate(
    input_variables=["summary", "language"],
    template="Translate the following summary into {language}:\n\n{summary}"
)

# Chains
summarize_chain = LLMChain(llm=llm, prompt=summary_prompt)
translate_chain = LLMChain(llm=llm, prompt=translate_prompt)

# Main function for Gradio
def summarize_and_translate_interface(text, language):
    summary = summarize_chain.run(text)
    translation = translate_chain.run({
        "summary": summary,
        "language": language
    })
    return summary, translation

# Gradio UI
iface = gr.Interface(
    fn=summarize_and_translate_interface,
    inputs=[
        gr.Textbox(lines=7, label="Enter Text to Summarize and Translate"),
        gr.Textbox(label="Target Language (e.g., French, Hindi, Tamil)")
    ],
    outputs=[
        gr.Textbox(label="🔍 Summary"),
        gr.Textbox(label="🌐 Translation")
    ],
    title="🧠 AI Text Summarizer & Translator",
    description="This app summarizes any input text and translates the summary into your chosen language using Groq's LLaMA3-8B model."
)

if __name__ == "__main__":
    iface.launch()


C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Load API Key
load_dotenv()

# Setup Groq LLM
llm = ChatGroq(
    temperature=0.3,
    model_name="llama3-8b-8192"
)

# --------------------------
# Agents
# --------------------------

# Researcher Agent: figures out the best algorithm and approach
researcher = Agent(
    role="Algorithm Researcher",
    goal="Determine the most optimal algorithm and data structure to solve coding problems",
    backstory=(
        "An expert in algorithm design, known for crafting high-performance solutions "
        "for competitive coding. Has deep knowledge of time complexity and algorithm selection."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# Coder Agent: writes efficient Python code
coder = Agent(
    role="Python Code Generator",
    goal="Generate fast, optimized Python code that runs in under 10ms for given problems",
    backstory=(
        "A seasoned software engineer who writes highly optimized code, leveraging "
        "advanced data structures and Python tricks to get minimal runtime on LeetCode."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# --------------------------
# Tasks
# --------------------------

# Task for researcher to determine optimal algorithm
research_task = Task(
    description=(
        "Analyze the following LeetCode problem and determine the best algorithm to solve it "
        "with time complexity ideally O(1), O(log n), or O(n). Consider edge cases.\n\n"
        "Problem: {user_problem}"
    ),
    expected_output="Detailed explanation of the best algorithm, time and space complexity, and pseudocode.",
    agent=researcher
)

# Task for coder to implement the solution
code_task = Task(
    description=(
        "Using the research summary, implement the optimal Python code for the following problem:\n\n"
        "{user_problem}\n\n"
        "Make sure the solution is correct, handles edge cases, and runs efficiently (<=10ms runtime)."
    ),
    expected_output="Well-commented, fast, and correct Python code with no unnecessary overhead.",
    agent=coder
)

# --------------------------
# Crew Assembly
# --------------------------

def solve_leetcode_problem(problem_text: str):
    # Inject problem into task descriptions
    research_task.description = research_task.description.format(user_problem=problem_text)
    code_task.description = code_task.description.format(user_problem=problem_text)

    crew = Crew(
        agents=[researcher, coder],
        tasks=[research_task, code_task],
        process=Process.sequential,
        verbose=True
    )

    result = crew.kickoff()
    print("\n🧠 Final Code Output:\n")
    print(result)

# --------------------------
# Example Use
# --------------------------

if __name__ == "__main__":
    problem = """
    Given an array of integers nums and an integer target, return indices of the two numbers 
    such that they add up to target. You may assume that each input would have exactly one solution, 
    and you may not use the same element twice.

    You can return the answer in any order.
    """
    solve_leetcode_problem(problem)
